In [1]:
# pip install --upgrade "ibm-watson>=5.3.0"
# Make sure your time/location range is the same as New York Time
# If on Windows, right click the date and time on the lower right.
# Click "Adjust Date/Time".
# Click "Sync now."
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import json
import requests
import webbrowser
from threading import Thread
import struct
import time
import hashlib
import base64
import socket
import time
import types
import multiprocessing
import os
import collections
import pandas as pd
%run PlaceSearch.ipynb

mode = "initialize"
preference = {}
using_watson = True
authenticator = IAMAuthenticator('qUj4Zg8prt9j2jOCIBamphxWFyYXkKS8QmYZk1E-n-mM')
assistant = AssistantV2(
    version='2021-11-11',
    authenticator=authenticator
)
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')
sessionID = assistant.create_session(
    assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
).get_result()
class returnCrossDomain(Thread):
    def __init__(self, connection):
        Thread.__init__(self)
        self.con = connection
        self.isHandleShake = False
    def run(self):
        global mode
        global preference
        global using_watson
        global authenticator
        global assistant
        global sessionID
        while True:
            if not self.isHandleShake:
                header = self.analyzeReq()
                secKey = header['Sec-WebSocket-Key']
                acceptKey = self.generateAcceptKey(secKey)
                response = "HTTP/1.1 101 Switching Protocols\r\n"
                response += "Upgrade: websocket\r\n"
                response += "Connection: Upgrade\r\n"
                response += "Sec-WebSocket-Accept: %s\r\n\r\n" % (acceptKey.decode('utf-8'))
                self.con.send(response.encode())
                self.isHandleShake = True
                if(mode=="initialize"):
                    mode = "get_order"
                print('response:\r\n' + response)
            elif mode == "get_order":
                opcode = self.getOpcode()
                if opcode == 8:
                    self.con.close()
                self.getDataLength()
                clientData = self.readClientData()
#                 print('Client：' + str(clientData))
                if str(clientData) == 'restart' or str(clientData) == 'Restart':
                    authenticator = IAMAuthenticator('qUj4Zg8prt9j2jOCIBamphxWFyYXkKS8QmYZk1E-n-mM')
                    assistant = AssistantV2(
                        version='2021-11-11',
                        authenticator=authenticator
                    )
                    assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')
                    sessionID = assistant.create_session(
                        assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
                    ).get_result()
                    using_watson = True
                if using_watson:
                    ans, choices, preference = self.answer(clientData)
                    if choices:
                        ans += '\n' + " or ".join(choices)
                    if "OK. I will start" in ans:
                        using_watson = False
                        print(preference)
                        
                    self.sendDataToClient(ans)
                if not using_watson:
                    self.sendDataToClient("Wating....... Please wait for a moment. Thanks for your patience!")
                    print(preference)
                    items = [preference["place"], "play"]
                    place = preference["place"]
                    requirement = {
                        preference["place"] : {
                            'price' : preference['price'] if 'price' in preference else 'no_require',
                            'rating' : preference['rating'] if 'rating' in preference else 'no_require',
                            "safety":"high"
                        }, 
                        "play": {"rating":"high", "safety":"high"}
                    }
                    
                    if place == 'hotel':
                        requirement['hotel']['bed'] = preference['bed'][1:-1] if 'bed' in preference else None
                    print(requirement)
                    output = recommendPlace(
                                    requirement,
                                    items, 
                                    preference['zipcode'], 
                                    5000,
                                    "2021-12-05",
                                    3
                                   )
                    ans = "Here is what we found \n"
                    ans += ' \n'
#                     max_address = max(max(len(i) for i in output[preference["place"]]['formatted_address']), 7)
#                     max_address = max_address if (max_address - 7) %2 ==0 else max_address + 1
#                     max_name = max(max(len(i) for i in output[preference["place"]]['name']), len(preference["place"] + " Name"))
#                     max_name = max_name if (max_name - len(preference["place"] + " Name")) % 2 == 0 else max_name + 1
#                     total_length = max_address + max_name + 3
#                     empty_dashes = '-' * total_length
#                     ans += empty_dashes + "\n"
#                     ans += '|' + " " * ((max_address - 9) // 2) + 'Address' + " " * ((max_address - 9) // 2) + '|' + " " * ((max_name - len(preference["place"] + " Name") - 2) // 2) + preference["place"] + " Name"  + " " * ((max_name - len(preference["place"] + " Name") - 2) // 2) + '|\n'
#                     ans += empty_dashes + "\n"
                    
                    for index, row in output[preference["place"]].iterrows():
#                         print(row)
                        address = row['formatted_address']
                        name = row['name']
                        ans += address + ', ' + name + '\n'
                        ans += '\n'
#                         ans +=  '|' + address + " " * (max_address - len(address)) + '|' + name  + " " * (max_name - len(name)) + '|\n'
#                         ans += empty_dashes + "\n"
        
                    ans += "\n We also found some places interesting for you! \n"
#                     max_address = max(max(len(i) for i in output['play']['formatted_address']), 7)
#                     max_address = max_address if (max_address - 7) %2 ==0 else max_address + 1
#                     max_name = max(max(len(i) for i in output['play']['name']), 4)
#                     max_name = max_name if (max_name - 4) % 2 == 0 else max_name + 1
#                     total_length = max_address + max_name + 3
#                     empty_dashes = '-' * total_length
#                     ans += empty_dashes + "\n"
#                     ans += '|' + " " * ((max_address - 7) // 2) + 'Address' + " " * ((max_address - 7) // 2) + '|' + " " * ((max_name - 4) // 2) + " Name"  + " " * ((max_name - 4) // 2) + '|\n'
#                     ans += empty_dashes + "\n"
                    for index, row in output['play'].iterrows():
                        address = row['formatted_address']
                        name = row['name']
                        ans += address + ', ' + name + '\n'
                        ans += "\n"
#                         ans +=  '|' + address + " " * (max_address - len(address)) + '|' + name  + " " * (max_name - len(name)) + '|\n'
#                         ans += empty_dashes + "\n"
#                     print(ans)

                    ans += "\n Here are recommendations for places for fun by analyzing Tweets online! \n"
    
                    for index, row in output['twitter'].iterrows():
                        name = row['acutal_name']
                        ans += name + '\n'
                        ans += '\n'
                    self.sendDataToClient(ans)
                    
                    preference = {}
                    

    def analyzeReq(self):
        reqData = self.con.recv(1024).decode()
        reqList = reqData.split('\r\n')
        headers = {}
        for reqItem in reqList:
            if ': ' in reqItem:
                unit = reqItem.split(': ')
                headers[unit[0]] = unit[1]
        return headers

    def generateAcceptKey(self, secKey):
        sha1 = hashlib.sha1()
        sha1.update((secKey + '258EAFA5-E914-47DA-95CA-C5AB0DC85B11').encode())
        sha1_result = sha1.digest()
        acceptKey = base64.b64encode(sha1_result)
        return acceptKey

    def getOpcode(self):
        first8Bit = self.con.recv(1)
        first8Bit = struct.unpack('B', first8Bit)[0]
        opcode = first8Bit & 0b00001111
        return opcode

    def getDataLength(self):
        second8Bit = self.con.recv(1)
        second8Bit = struct.unpack('B', second8Bit)[0]
        masking = second8Bit >> 7
        dataLength = second8Bit & 0b01111111
        #print("dataLength:",dataLength)
        if dataLength <= 125:
            payDataLength = dataLength
        elif dataLength == 126:
            payDataLength = struct.unpack('H', self.con.recv(2))[0]
        elif dataLength == 127:
            payDataLength = struct.unpack('Q', self.con.recv(8))[0]
        self.masking = masking
        self.payDataLength = payDataLength

    def readClientData(self):

        if self.masking == 1:
            maskingKey = self.con.recv(4)
        data = self.con.recv(self.payDataLength)

        if self.masking == 1:
            i = 0
            trueData = ''
            for d in data:
                trueData += chr(d ^ maskingKey[i % 4])
                i += 1
            return trueData
        else:
            return data

    def sendDataToClient(self, text):
        sendData = ''
        sendData = struct.pack('!B', 0x81)

        length = len(text)
        if length <= 125:
            sendData += struct.pack('!B', length)
        elif length <= 65536:
            sendData += struct.pack('!B', 126)
            sendData += struct.pack('!H', length)
        elif length == 127:
            sendData += struct.pack('!B', 127)
            sendData += struct.pack('!Q', length)
        sendData += struct.pack('!%ds' % (length), text.encode())
        print(sendData)
        dataSize = self.con.sendall(sendData)

    def answer(self,data):
        entity_choices = ['restaurant', 'hotel']
        feature_choices = ['rating', 'Price', 'Room_type']
        return reply(data, entity_choices, feature_choices, preference)
    
def reply(input_data, entity_choices, feature_choices, preference):
    response = assistant.message(
    assistant_id='46543a05-d01d-4da7-a729-6daa8cb1e5b4',
    session_id=sessionID["session_id"],
        input={
        'message_type': 'text',
        'text': input_data
        }
    ).get_result()
    choice = False
    zipcode = ""
    text = response["output"]["generic"][0]["text"]
    intent = response["output"]["intents"][0]["intent"] if response["output"]["intents"] else ""
#     print(json.dumps(response, indent=2))
    choices = []
    if len(response["output"]['generic']) > 1 and "options" in response["output"]['generic'][1]:
        choice = True
        for opt in response["output"]['generic'][1]["options"]:
            choices.append(opt["label"])

    if intent in entity_choices:
        preference["place"] = intent
    elif intent in feature_choices:
        if intent == "Price":
            print(input_data)
            preference["price"] = "high" if 'Expensive' in input_data else "low"
        elif intent == "rating":
            preference['rating'] = "low" if input_data == "I do not care much about rating" else "high"
        elif intent == 'Room_type':
            preference['bed'] = input_data
    if response["output"]['entities']:
        if response["output"]['entities'][0]['entity'] == 'zipcode':
            zipcode = response["output"]['entities'][0]['value']
            preference['zipcode'] = input_data
    if not choice:
        return text, None, preference
    else:
        return text, choices, preference
    
def main():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('127.0.0.1', 9996))
    sock.listen(5)
    webbrowser.open("main.html", new = 2)
    while True:
        try:
            connection, address = sock.accept()
            returnCrossDomain(connection).start()
           
        except:
            time.sleep(1)

load Hotel Price


Downloading: 100%|███████████████████████████████████████████████████████████████| 278/278 [00:00<00:00, 1144.38rows/s]
C:\Users\JIANGA~1\AppData\Local\Temp/ipykernel_18632/4112926059.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_play["zipcode"] = [ad.split(" ")[-1] for ad in tmp_play["formatted_address"]]


In [ ]:
if __name__ == "__main__":
    main()

response:
HTTP/1.1 101 Switching Protocols
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Accept: bQOb+nO1W9bdI92wUKbTifSTSmI=


b"\x81nHi, this is Group 28's Project. What do you need to day? Please enter the zip code of the place you want to go"
b'\x81OSorry. Please enter a valid New York City zip code of the place you want to go.'
b'\x81OSorry. Please enter a valid New York City zip code of the place you want to go.'
b'\x81OSorry. Please enter a valid New York City zip code of the place you want to go.'
b'\x81ISure. Please tell me do you want to inquire about a restaurant or a hotel'
b'\x817Sure. Do you have any requirements for a place to eat? '
b'\x81ISure, Please select your preference about price below.\nCheap or Expensive'
 Expensive
b'\x81~\x00\xd0We do have a lot of restaurants in our system! Do you have any requirements for the rating of the restaurant? Please select your preference about rating below.\nI do not care much about rating or A good rating'
{'zipcode': '111

Downloading: 100%|███████████████████████████████████████████████████████████████| 278/278 [00:00<00:00, 1289.98rows/s]
C:\Users\JIANGA~1\AppData\Local\Temp/ipykernel_18632/4112926059.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_play["zipcode"] = [ad.split(" ")[-1] for ad in tmp_play["formatted_address"]]


b'\x81~\x01\xe7Here is what we found \n \n1240 6th Ave, New York, NY 10020, Magnolia Bakery - Rockefeller Center\n\n4610 Center Blvd, Queens, NY 11109, Maiella\n\n45-40 Center Blvd, Queens, NY 11109, Blend on the Water\n\n\n We also found some places interesting for you! \n1000 5th Ave, New York, NY 10028, The Metropolitan Museum of Art\n\nNew York, NY 10018, Bryant Park\n\n53 E 79th St, New York, NY 10075, The New York Society Library\n\n\n Here are recommendations for places for fun by analyzing Tweets online! \n'
b'\x81PHello. How can I help you? Please enter the zip code of the place you want to go'
b'\x81ISure. Please tell me do you want to inquire about a restaurant or a hotel'
b'\x817Sure. Do you have any requirements for a place to live?'
b'\x81ISure, Please select your preference about price below.\nCheap or Expensive'
 Expensive
b'\x81~\x00\xc6We do have a lot of hotels in our system! Do you have any requirements for the rating of the hotel? Please select your preference abou

Downloading: 100%|███████████████████████████████████████████████████████████████| 278/278 [00:00<00:00, 1040.17rows/s]


b'\x81~\x02\x1dHere is what we found \n \n2175 Broadway, New York, NY 10024, Hotel Belleclaire\n\n25 W 51st St, New York, NY 10019, Radisson Hotel New York Midtown-Fifth Avenue\n\n59 W 46th St, New York, NY 10036, The Hotel @ Times Square\n\n\n We also found some places interesting for you! \n1000 5th Ave, New York, NY 10028, The Metropolitan Museum of Art\n\nNew York, NY 10018, Bryant Park\n\n53 E 79th St, New York, NY 10075, The New York Society Library\n\n\n Here are recommendations for places for fun by analyzing Tweets online! \nCathedral of St. John the Divine\n\n'
b"\x81nHi, this is Group 28's Project. What do you need to day? Please enter the zip code of the place you want to go"
b'\x81ISure. Please tell me do you want to inquire about a restaurant or a hotel'
b'\x817Sure. Do you have any requirements for a place to eat? '
{'zipcode': '11101', 'place': 'restaurant'}
b'\x81GOK. I will start finding restaurants for you! Please wait for a moment.'
b'\x81AWating....... Please wait 

Downloading: 100%|███████████████████████████████████████████████████████████████| 278/278 [00:00<00:00, 1270.84rows/s]


b'\x81~\x01\xefHere is what we found \n \n1240 6th Ave, New York, NY 10020, Magnolia Bakery - Rockefeller Center\n\n45-40 Center Blvd, Queens, NY 11109, Blend on the Water\n\n308 E 78th St, New York, NY 10075, Orwashers Bakery\n\n\n We also found some places interesting for you! \n1000 5th Ave, New York, NY 10028, The Metropolitan Museum of Art\n\nNew York, NY 10018, Bryant Park\n\n53 E 79th St, New York, NY 10075, The New York Society Library\n\n\n Here are recommendations for places for fun by analyzing Tweets online! \n'
